In [2]:
ls

90803-project-s25.pdf                Data prep and baseline models.ipynb
Data/                                EDA.ipynb


In [3]:
cd '/Users/garrettkent/Desktop/Spring 2025/Machine Learning with Python/Final Project/Data'

/Users/garrettkent/Desktop/Spring 2025/Machine Learning with Python/Final Project/Data


In [4]:
ls

EDA.ipynb         Stop Usage.csv    stop_usage.csv
OTP by Route.csv  ridership.csv


In [5]:
import pandas as pd

# Load datasets
ridership = pd.read_csv("Ridership.csv")
stop_usage = pd.read_csv("Stop Usage.csv")

In [20]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

In [21]:
ridership.head()

,_id,route,ridership_route_code,route_full_name,current_garage,mode,month_start,year_month,day_type,avg_riders,day_count,total_precip,avg_temp
0,1,1,001,1 - FREEPORT ROAD,Ross,Bus,2017-01-01,201701,SAT.,969.50,4,3.54,34.6
1,2,4,004,4 - TROY HILL,Ross,Bus,2017-01-01,201701,SAT.,218.25,4,3.54,34.6
2,3,6,006,6 - SPRING HILL,Ross,Bus,2017-01-01,201701,SAT.,495.50,4,3.54,34.6
3,4,8,008,8 - PERRYSVILLE,Ross,Bus,2017-01-01,201701,SAT.,1480.00,4,3.54,34.6
4,5,11,011,11 - FINEVIEW,Ross,Bus,2017-01-01,201701,SAT.,208.00,4,3.54,34.6


In [22]:
X = ridership.drop(['avg_riders', 'route_full_name'], axis=1)
X.head()

,_id,route,ridership_route_code,current_garage,mode,month_start,year_month,day_type,day_count,total_precip,avg_temp
0,1,1,001,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
1,2,4,004,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
2,3,6,006,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
3,4,8,008,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
4,5,11,011,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6


In [23]:
y = ridership['avg_riders']
y.head()

0     969.50
1     218.25
2     495.50
3    1480.00
4     208.00
Name: avg_riders, dtype: float64

In [24]:
X.shape, y.shape

((22317, 11), (22317,))

In [25]:
# identifying null values and dropping sparse columns

In [26]:
X.isna().sum()

_id                         0
route                       1
ridership_route_code    10938
current_garage            189
mode                        1
month_start                 0
year_month                  0
day_type                    0
day_count                   0
total_precip                0
avg_temp                 2434
dtype: int64

In [27]:
X.nunique(axis=0)

_id                     22317
route                     108
ridership_route_code      121
current_garage              6
mode                        4
month_start                94
year_month                 94
day_type                    3
day_count                   9
total_precip               85
avg_temp                   80
dtype: int64

In [28]:
X.nunique()/len(X)*100

_id                     100.000000
route                     0.483936
ridership_route_code      0.542188
current_garage            0.026885
mode                      0.017924
month_start               0.421204
year_month                0.421204
day_type                  0.013443
day_count                 0.040328
total_precip              0.380876
avg_temp                  0.358471
dtype: float64

In [29]:
sparse_columns = list(X.columns[X.nunique()/len(X)*100 < 0.01])
X = X.drop(sparse_columns, axis=1)

In [30]:
#imputation, one-hot encoding, and training baseline models with cross-validation

In [31]:
X.head()

,_id,route,ridership_route_code,current_garage,mode,month_start,year_month,day_type,day_count,total_precip,avg_temp
0,1,1,001,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
1,2,4,004,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
2,3,6,006,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
3,4,8,008,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6
4,5,11,011,Ross,Bus,2017-01-01,201701,SAT.,4,3.54,34.6


In [32]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import pandas as pd

# 1. Separate columns by dtype
int_cols = X.select_dtypes(include='int').columns.tolist()
float_cols = X.select_dtypes(include='float').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

# 2. Define preprocessing for numeric and categorical data
numeric_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Combine into one column transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_preprocessor, int_cols + float_cols),
    ('cat', categorical_preprocessor, cat_cols)
])

# 4. Define models
models = {
    "Baseline (Mean)": DummyRegressor(strategy='mean'),
    "Linear Regression": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]),
    "Random Forest": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=25, random_state=47, n_jobs=-1))
    ])
}

# 5. Cross-validate and evaluate
cv = KFold(n_splits=5, shuffle=True, random_state=47)

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
    rmse_scores = -scores
    print(f"{name} | Mean RMSE: {rmse_scores.mean():.3f} | Std: {rmse_scores.std():.3f}")

Baseline (Mean) | Mean RMSE: 1393.608 | Std: 39.015
Linear Regression | Mean RMSE: 694.097 | Std: 27.493
Random Forest | Mean RMSE: 235.160 | Std: 12.546


In [33]:
stop_usage.head()

,_id,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,datekey,time_period,route_name,serviceday,total_ons,total_offs,days,avg_ons,avg_offs
0,1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sat,12.0,0.0,4,3.000000,0.00
1,2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sun,14.0,0.0,6,2.333333,0.00
2,3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Weekday,64.0,1.0,20,3.200000,0.05
3,4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,P69,Weekday,39.0,0.0,20,1.950000,0.00
4,5,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,202001,Pre-pandemic,69,Sat,11.0,0.0,4,2.750000,0.00


In [34]:
X = stop_usage.iloc[:,:-5]
X.head()

,_id,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,datekey,time_period,route_name,serviceday
0,1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sat
1,2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sun
2,3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Weekday
3,4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,P69,Weekday
4,5,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,202001,Pre-pandemic,69,Sat


In [35]:
y = stop_usage.iloc[:,-5:]

In [36]:
y.head()

,total_ons,total_offs,days,avg_ons,avg_offs
0,12.0,0.0,4,3.000000,0.00
1,14.0,0.0,6,2.333333,0.00
2,64.0,1.0,20,3.200000,0.05
3,39.0,0.0,20,1.950000,0.00
4,11.0,0.0,4,2.750000,0.00


In [37]:
y = y.iloc[:,0]
y.head()

0    12.0
1    14.0
2    64.0
3    39.0
4    11.0
Name: total_ons, dtype: float64

In [38]:
y.isna().sum()

206

In [39]:
X.shape, y.shape

((107611, 15), (107611,))

In [40]:
X = X[~y.isna()]
y = y[~y.isna()]

In [41]:
#X.nunique(axis=0)

In [42]:
#X.nunique()/len(X)*100

In [43]:
#sparse_columns = list(X.columns[X.nunique()/len(X)*100 < 0.01])
#X = X.drop(sparse_columns, axis=1)

In [44]:
# random forest performed way worse here without the sparse columns

In [45]:
# 1. Separate columns by dtype
int_cols = X.select_dtypes(include='int').columns.tolist()
float_cols = X.select_dtypes(include='float').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

# 2. Define preprocessing for numeric and categorical data
numeric_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Combine into one column transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_preprocessor, int_cols + float_cols),
    ('cat', categorical_preprocessor, cat_cols)
])

# 4. Define models
models = {
    "Baseline (Mean)": DummyRegressor(strategy='mean'),
    "Linear Regression": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]),
    "Random Forest": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=10, random_state=47, n_jobs=-1))
    ])
}

# 5. Cross-validate and evaluate
cv = KFold(n_splits=5, shuffle=True, random_state=47)

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
    rmse_scores = -scores
    print(f"{name} | Mean RMSE: {rmse_scores.mean():.3f} | Std: {rmse_scores.std():.3f}")

Baseline (Mean) | Mean RMSE: 378.270 | Std: 54.625
Linear Regression | Mean RMSE: 321.806 | Std: 52.104
Random Forest | Mean RMSE: 189.339 | Std: 42.926


In [46]:
def count_outliers_iqr(df, cols=None, multiplier=1.5):
    """
    Count the number of outliers in each numeric column using the IQR method.

    Parameters:
    - df: pandas DataFrame
    - cols: list of columns to check (default: all numeric)
    - multiplier: controls sensitivity (1.5 is standard)

    Returns:
    - pandas Series with outlier counts per column
    """
    if cols is None:
        cols = df.select_dtypes(include=['int', 'float']).columns

    outlier_counts = {}

    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - multiplier * IQR
        upper = Q3 + multiplier * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        outlier_counts[col] = len(outliers)

    return pd.Series(outlier_counts).sort_values(ascending=False)

In [47]:
count_outliers_iqr(X)

latitude     8338
longitude    2187
clever_id     907
_id             0
datekey         0
dtype: int64

In [48]:
X = ridership.drop(['avg_riders', 'route_full_name'], axis=1)
count_outliers_iqr(X)

total_precip    475
_id               0
year_month        0
day_count         0
avg_temp          0
dtype: int64